# Lesson 4: Comprehensive Testing Framework

In [1]:
import warnings
warnings.filterwarnings('ignore')

## Import the API keys

In [2]:
from utils import get_circle_api_key
cci_api_key = get_circle_api_key()

In [3]:
from utils import get_gh_api_key
gh_api_key = get_gh_api_key()

In [4]:
from utils import get_openai_api_key
openai_api_key = get_openai_api_key()

## Set up our github branch

In [5]:
from utils import get_repo_name
course_repo = get_repo_name()
course_repo

'CircleCI-Learning/llmops-course'

In [6]:
from utils import get_branch
course_branch = get_branch()
course_branch

'dl-cci-snappy-client-62'

Aditionally, the file quiz_bank.txt should be uploaded to this workspace (already done!). This file has the information about the content we want for the quizzes generated.

In [7]:
from utils import read_file_into_string
quiz_bank = read_file_into_string("quiz_bank.txt")

In [8]:
print(quiz_bank)

1. Subject: Leonardo DaVinci
   Categories: Art, Science
   Facts:
    - Painted the Mona Lisa
    - Studied zoology, anatomy, geology, optics
    - Designed a flying machine
  
2. Subject: Paris
   Categories: Art, Geography
   Facts:
    - Location of the Louvre, the museum where the Mona Lisa is displayed
    - Capital of France
    - Most populous city in France
    - Where Radium and Polonium were discovered by scientists Marie and Pierre Curie

3. Subject: Telescopes
   Category: Science
   Facts:
    - Device to observe different objects
    - The first refracting telescopes were invented in the Netherlands in the 17th Century
    - The James Webb space telescope is the largest telescope in space. It uses a gold-berillyum mirror

4. Subject: Starry Night
   Category: Art
   Facts:
    - Painted by Vincent van Gogh in 1889
    - Captures the east-facing view of van Gogh's room in Saint-Rémy-de-Provence

5. Subject: Physics
   Category: Science
   Facts:
    - The sun doesn't chan

## Rebuild the quiz generator app

In [9]:
delimiter = "####"
system_message = f"""
Follow these steps to generate a customized quiz for the user.
The question will be delimited with four hashtags i.e {delimiter}

Step 1:{delimiter} First identify the category user is asking about from the following list:
* Geography
* Science
* Art

Step 2:{delimiter} Determine the subjects to generate questions about. The list of topics are below:

{quiz_bank}

Pick up to two subjects that fit the user's category.

Step 3:{delimiter} Generate a quiz for the user. Based on the selected subjects generate 3 questions for the user using the facts about the subject.
* Include any facts that might be interesting

Use the following format:
Question 1:{delimiter} <question 1>

Question 2:{delimiter} <question 2>

Question 3:{delimiter} <question 3>
"""

## Create Chat Prompt
Create function assistant_chain() to build the chat_prompt.

In [10]:
from langchain.prompts                import ChatPromptTemplate
from langchain.chat_models            import ChatOpenAI
from langchain.schema.output_parser   import StrOutputParser

def assistant_chain():
  human_template  = "{question}"

  chat_prompt = ChatPromptTemplate.from_messages([
      ("system", system_message),
      ("human", human_template),
  ])
  return chat_prompt | \
         ChatOpenAI(model="gpt-3.5-turbo", 
                    temperature=0) | \
         StrOutputParser()

# Hallucination detection

In [11]:
from langchain.prompts                import ChatPromptTemplate
from langchain.chat_models            import ChatOpenAI
from langchain.schema.output_parser   import StrOutputParser

def create_eval_chain(context, agent_response):
  eval_system_prompt = """You are an assistant that evaluates \
  how well the quiz assistant
    creates quizzes for a user by looking at the set of \
    facts available to the assistant.
    Your primary concern is making sure that ONLY facts \
    available are used. Quizzes that contain facts outside
    the question bank are BAD quizzes and harmful to the student."""
  
  eval_user_message = """You are evaluating a generated quiz \
  based on the context that the assistant uses to create the quiz.
  Here is the data:
    [BEGIN DATA]
    ************
    [Question Bank]: {context}
    ************
    [Quiz]: {agent_response}
    ************
    [END DATA]

Compare the content of the submission with the question bank \
using the following steps

1. Review the question bank carefully. \
  These are the only facts the quiz can reference
2. Compare the quiz to the question bank.
3. Ignore differences in grammar or punctuation
4. If a fact is in the quiz, but not in the question bank \
   the quiz is bad.

Remember, the quizzes need to only include facts the assistant \
  is aware of. It is dangerous to allow made up facts.

Output Y if the quiz only contains facts from the question bank, \
output N if it contains facts that are not in the question bank.
"""
  eval_prompt = ChatPromptTemplate.from_messages([
      ("system", eval_system_prompt),
      ("human", eval_user_message),
  ])

  return eval_prompt | ChatOpenAI(
      model="gpt-3.5-turbo", 
      temperature=0) | \
    StrOutputParser()

Create function test_model_graded_eval_hallucination(quiz_bank) to test about hallucinations.

In [12]:
def test_model_graded_eval_hallucination(quiz_bank):
  assistant = assistant_chain()
  quiz_request = "Write me a quiz about books."
  result = assistant.invoke({"question": quiz_request})
  print(result)
  eval_agent = create_eval_chain(quiz_bank, result)
  eval_response = eval_agent.invoke({"context": quiz_bank, "agent_response": result})
  print(eval_response)
  # Our test asks about a subject not in the context, so the agent should answer N
  assert eval_response == "N"


Try test_model_graded_eval_hallucination function with the quiz_bank (mentioned before).

In [13]:
test_model_graded_eval_hallucination(quiz_bank)

#### First identify the category user is asking about from the following list:
* Geography
* Science
* Art

Since you mentioned books, I will assume you are interested in Art.

#### Determine the subjects to generate questions about. The list of topics are below:

1. Subject: Leonardo DaVinci
   Categories: Art, Science
   Facts:
    - Painted the Mona Lisa
    - Studied zoology, anatomy, geology, optics
    - Designed a flying machine
  
2. Subject: Paris
   Categories: Art, Geography
   Facts:
    - Location of the Louvre, the museum where the Mona Lisa is displayed
    - Capital of France
    - Most populous city in France
    - Where Radium and Polonium were discovered by scientists Marie and Pierre Curie

3. Subject: Telescopes
   Category: Science
   Facts:
    - Device to observe different objects
    - The first refracting telescopes were invented in the Netherlands in the 17th Century
    - The James Webb space telescope is the largest telescope in space. It uses a gold-berill

## Reviewing evaluations

Improve the eval_system_prompt.

In [14]:
eval_system_prompt = """You are an assistant that evaluates \
how well the quiz assistant
    creates quizzes for a user by looking at the set of \
    facts available to the assistant.
    Your primary concern is making sure that ONLY facts \
    available are used.
    Helpful quizzes only contain facts in the test set."""

Rebuild the evaluator to provide not only the decision of the result but also the explanation. 

In [15]:
eval_user_message = """You are evaluating a generated quiz based on the question bank that the assistant uses to create the quiz.
  Here is the data:
    [BEGIN DATA]
    ************
    [Question Bank]: {context}
    ************
    [Quiz]: {agent_response}
    ************
    [END DATA]

## Examples of quiz questions
Subject: <subject>
   Categories: <category1>, <category2>
   Facts:
    - <fact 1>
    - <fact 2>

## Steps to make a decision
Compare the content of the submission with the question bank using the following steps

1. Review the question bank carefully. These are the only facts the quiz can reference
2. Compare the information in the quiz to the question bank.
3. Ignore differences in grammar or punctuation

Remember, the quizzes should only include information from the question bank.


## Additional rules
- Output an explanation of whether the quiz only references information in the context.
- Make the explanation brief only include a summary of your reasoning for the decsion.
- Include a clear "Yes" or "No" as the first paragraph.
- Reference facts from the quiz bank if the answer is yes

Separate the decision and the explanation. For example:

************
Decision: <Y>
************
Explanation: <Explanation>
************
"""

Rebuild the check prompt template with the new prompt.

In [16]:
eval_prompt = ChatPromptTemplate.from_messages([
      ("system", eval_system_prompt),
      ("human", eval_user_message),
  ])
eval_prompt

ChatPromptTemplate(input_variables=['agent_response', 'context'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an assistant that evaluates how well the quiz assistant\n    creates quizzes for a user by looking at the set of     facts available to the assistant.\n    Your primary concern is making sure that ONLY facts     available are used.\n    Helpful quizzes only contain facts in the test set.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_response', 'context'], template='You are evaluating a generated quiz based on the question bank that the assistant uses to create the quiz.\n  Here is the data:\n    [BEGIN DATA]\n    ************\n    [Question Bank]: {context}\n    ************\n    [Quiz]: {agent_response}\n    ************\n    [END DATA]\n\n## Examples of quiz questions\nSubject: <subject>\n   Categories: <category1>, <category2>\n   Facts:\n    - <fact 1>\n    - <fact 2>\n\n## Steps to make a 

### Building multiple test cases to use the evaluator and new prompt

Define different test cases, prompts that include inputs and their expected responses.


*_Note_*: Try adding your own inputs and responses!

In [17]:
# In a real application you would load your dataset from a file or logging tool.
# Here we have a mix of examples with slightly different phrasing that our quiz application can support
# and things we don't support. 
test_dataset = [
  {"input": "I'm trying to learn about science, can you give me a quiz to test my knowledge",
   "response": "science",
   "subjects": ["davinci", "telescope", "physics", "curie"]},
  {"input": "I'm an geography expert, give a quiz to prove it?",
   "response": "geography",
   "subjects": ["paris", "france", "louvre"]},
   {"input": "Quiz me about Italy",
   "response": "geography",
   "subjects": ["rome", "alps", "sicily"]
   },
]

Create funcion evaluate_dataset to look through the dataset and invoke the quiz generator.

In [18]:
def evaluate_dataset(dataset, 
                     quiz_bank,
                     assistant, 
                     evaluator):
  eval_results = []
  for row in dataset:
    eval_result = {}
    user_input = row["input"] 
    answer = assistant.invoke({"question": user_input})
    eval_response = evaluator.invoke({"context": quiz_bank, "agent_response": answer})
    
    eval_result["input"] = user_input
    eval_result["output"] = answer
    eval_result["grader_response"] = eval_response
    eval_results.append(eval_result)
  return eval_results

Import functions to generate reports for the test_dataset evaluations.

In [19]:
import pandas as pd
from app import assistant_chain, quiz_bank
from IPython.display import display, HTML

from langchain.prompts                import ChatPromptTemplate
from langchain.chat_models            import ChatOpenAI
from langchain.schema.output_parser   import StrOutputParser

Create the wrapper, incluide the LLM (in this case GTP-3.5-turbo).

In [20]:
def create_eval_chain(prompt):
  
  return prompt | \
    ChatOpenAI(model="gpt-3.5-turbo", 
               temperature=0) | \
    StrOutputParser()


Use pandas to facilitate the creation of reports and define report_evals() function.

In [21]:
def report_evals(display_to_notebook=False):
  assistant = assistant_chain()
  model_graded_evaluator = create_eval_chain(eval_prompt)
  eval_results = evaluate_dataset(test_dataset, 
                                  quiz_bank, 
                                  assistant, 
                                  model_graded_evaluator)
  df = pd.DataFrame(eval_results)
  ## clean up new lines to be html breaks
  df_html = df.to_html().replace("\\n","<br>")
  if display_to_notebook:
    display(HTML(df_html))
  else:
    print(df_html)


Display the table with the results (grader_response).

<p style="background-color:#ffc107; padding:15px; margin-left:20px"> <b>Note:</b> Depending on the specific wording of the questions generated by your application, you may receive a different decision from the evaluation model in the following cell. This is a good example of why storing evaluation results for human review can be useful in refining model behavior.</p>


In [22]:
report_evals(display_to_notebook=True)

,input,output,grader_response
0,"I'm trying to learn about science, can you give me a quiz to test my knowledge","Step 1:#### ScienceStep 2:#### Subjects: Telescopes, PhysicsStep 3:#### Generate a quiz for the user:Question 1:#### What is the James Webb space telescope known for?Question 2:#### True or False: Water slows the speed of light.Question 3:#### Why is the Eiffel Tower in Paris taller in the summer than the winter?","Decision: NoExplanation: The quiz includes a question about the Eiffel Tower in Paris, which is not a subject or fact provided in the question bank. Additionally, the question about the speed of light and water is not directly related to the facts in the question bank about Physics."
1,"I'm an geography expert, give a quiz to prove it?","Step 1:#### GeographyStep 2:#### ParisStep 3:#### Generate a quiz for the user.Question 1:#### What is the capital of France?Question 2:#### Where can you find the Louvre, the museum where the Mona Lisa is displayed?Question 3:#### Which famous scientists discovered Radium and Polonium in Paris?","Decision: No************Explanation: The quiz includes information about the capital of France, the location of the Louvre museum, and the discovery of Radium and Polonium in Paris, which are not present in the question bank. Therefore, the quiz contains facts outside the provided data."
2,Quiz me about Italy,"#### Step 1:####The user is asking about Geography.#### Step 2:####Subjects related to Italy:1. Subject: Leonardo DaVinci Categories: Art, Science2. Subject: Paris Categories: Art, Geography3. Subject: Starry Night Category: Art#### Step 3:####Question 1:#### In which city is the Louvre located, the museum where the Mona Lisa is displayed?Question 2:#### Who painted the famous artwork ""Starry Night"" in 1889?Question 3:#### Which artist studied zoology, anatomy, geology, and optics, and designed a flying machine?","Decision: No************Explanation: The quiz includes a question about the artist who painted ""Starry Night,"" which is not a subject or fact provided in the question bank. Additionally, the question about the Louvre's location, while indirectly related to Paris, does not directly align with the available facts about Paris in the question bank."


## Integration to CircleCI

Display the additional file for the CI/CD integrations.

In [23]:
!cat save_eval_artifacts.py

import pandas as pd
from app import assistant_chain, quiz_bank
from IPython.display import display, HTML

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

eval_system_prompt = """You are an assistant that evaluates how well the quiz assistant
    creates quizzes for a user by looking at the set of facts available to the assistant.
    Your primary concern is making sure that ONLY facts available are used. Helpful quizzes only contain facts in the
    test set"""

eval_user_message = """You are evaluating a generated quiz based on the question bank that the assistant uses to create the quiz.
  Here is the data:
    [BEGIN DATA]
    ************
    [Question Bank]: {context}
    ************
    [Quiz]: {agent_response}
    ************
    [END DATA]

## Examples of quiz questions
Subject: <subject>
   Categories: <category1>, <category2>
   Facts:
    - <fact 1>
    - <fact 2>

## 

Import the function trigger_eval_report adding our recent configurations and then at CircleCI website you will see the evaluation.

In [24]:
from utils import trigger_eval_report
trigger_eval_report(course_repo, 
                    course_branch, 
                    ["app.py", 
                     "save_eval_artifacts.py",
                     "quiz_bank.txt"], 
                    cci_api_key)

uploading quiz_bank.txt
uploading save_eval_artifacts.py
uploading app.py
Please visit https://app.circleci.com/pipelines/github/CircleCI-Learning/llmops-course/6212
